In [1]:
from enum import Enum

class Side(Enum):
    BUY = 1
    SELL = -1

class Status(Enum):
    OPEN = 0
    BE = 1
    WIN = 2
    LOSS = 3

class Execution:
    def __init__(self, side, traded_price, traded_quantity, execution_date, fees=0, commission=0):
        self.side = side
        self.traded_price = traded_price
        self.traded_quantity = traded_quantity
        self.execution_date = execution_date
        self.fees = fees
        self.value = traded_price * traded_quantity
        self.commission = commission
    
    def data(self):
        data = dict()
        data['execution_date'] = self.execution_date
        data['side'] = self.side.name
        data['traded_price'] = self.traded_price
        data['traded_quantity'] = self.traded_quantity
        data['value'] = self.value
        data['fees'] = self.fees
        data['commission'] = self.commission
        return data
    
    def __str__(self):
        return f'{self.side}, {self.traded_price}, {self.traded_quantity}'


class Order:
    def __init__(self, ticker, execution):
        self.ticker = ticker
        self.net_position = 0  # current size
        self.avg_open_price = 0
        self.net_investment = 0
        self.realized_pnl = 0
        self.unrealized_pnl = 0
        self.total_pnl = 0
        self.is_open = True
        self.executions = []
        self.max_size = 0
        self.status = Status.OPEN
        self.total_fees = 0
        self.total_commissions = 0
        self.update_by_tradefeed(execution)
        
    def update_by_tradefeed(self, execution):
        self.executions.append(execution)
        self.total_fees += execution.fees
        self.total_commissions += execution.commission
        
        # buy: positive position, sell: negative position
        quantity_with_direction = execution.side.value * execution.traded_quantity
    
        self.is_open = (self.net_position * quantity_with_direction) >= 0
        
        # net investment
        self.net_investment = max( self.net_investment, abs( self.net_position * self.avg_open_price  ) )
        
        # realized pnl
        if not self.is_open:
            # Remember to keep the sign as the net position
            self.realized_pnl += ( execution.traded_price - self.avg_open_price ) *  min( 
                    abs(quantity_with_direction), 
                    abs(self.net_position) 
                ) * ( abs(self.net_position) / self.net_position )
            
        # total pnl
        self.total_pnl = self.realized_pnl + self.unrealized_pnl
        
        # avg open price
        if self.is_open:
            self.avg_open_price = ( ( self.avg_open_price * self.net_position ) + 
                ( execution.traded_price * quantity_with_direction ) ) / ( self.net_position + quantity_with_direction )
        else:
            # Check if it is close-and-open
            if execution.traded_quantity > abs(self.net_position):
                self.avg_open_price = execution.traded_price
            else:
                self.status = Status.WIN if self.total_pnl > 0 else (
                              Status.LOSS if self.total_pnl < 0 else Status.BE
                )
        
        # net position
        self.net_position += quantity_with_direction
        
        self.max_size = max(self.max_size, self.net_position)

    def update_by_marketdata(self, last_price):
        self.unrealized_pnl = ( last_price - self.avg_open_price ) * self.net_position
        self.total_pnl = self.realized_pnl + self.unrealized_pnl
    
    def data(self):
        data = dict()
        data['status'] = self.status.name
        data['size'] = self.max_size
        data['cost'] = self.max_size * self.avg_open_price
        data['avg_open_price'] = self.avg_open_price
        data['avg_close_price'] = self.avg_open_price + self.total_pnl / self.max_size if self.net_position == 0 else None
        data['return'] = self.total_pnl if self.net_position == 0 else None
        data['return_percent'] = self.total_pnl / (self.max_size * self.avg_open_price) if self.net_position == 0 else None
        data['total_commissions'] = self.total_commissions
        data['total_fees'] = self.total_fees
        return data

In [2]:
order = Order('AAPL', Execution(Side.BUY, 109.75, 10, None))
order.net_position

10

In [3]:
order.update_by_tradefeed(Execution(Side.BUY, 110, 10, None))

In [4]:
order.update_by_tradefeed(Execution(Side.SELL, 111, 20, None))

In [5]:
order.total_pnl

22.5

In [6]:
order.is_open

False

In [7]:
order.unrealized_pnl

0

In [8]:
order.executions[0]

In [9]:
order.total_pnl

22.5

In [10]:
order.avg_open_price

109.875

In [11]:
(110*10 + 109.75*10)/20

109.875

In [12]:
import pandas as pd

pd.DataFrame([e.data() for e in order.executions])

,commission,execution_date,fees,side,traded_price,traded_quantity,value
0,0,None,0,BUY,109.75,10,1097.5
1,0,None,0,BUY,110.00,10,1100.0
2,0,None,0,SELL,111.00,20,2220.0


In [13]:
order_df = pd.DataFrame([order.data()])
display(order_df)

,avg_close_price,avg_open_price,cost,return,return_percent,size,status,total_commissions,total_fees
0,111.0,109.875,2197.5,22.5,0.010239,20,WIN,0,0


In [14]:
order.is_open

False